# SSW2024 Workshop 1 HCIPy Handout

## Author: Emiel Por (Space Telescope Science Institute)

Run the `SSW2024_HCIPy_Setup` notebook to install the required packages and download the data. **TBD - A setup notebook only needed if there is data to install as well.**

## Introduction

As you've heard during the morning session, coronagraphs are an integral part of any high-contrast imaging system. This session is meant to demonstrate how numerical simulations

Install `HCIPy` and `tqdm`

In [ ]:
# Install HCIPy and tqdm
!pip install hcipy
!pip install tqdm

In [ ]:
import hcipy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
def make_animation(f, parameter_name, parameter_values, framerate=5, **kwargs):
    mw = hcipy.FFMpegWriter(f'{f.__name__}_{parameter_name}.mp4', framerate=framerate)

    for p in tqdm(parameter_values):
        plt.clf()

        f(**{parameter_name: p}, **kwargs)
        plt.suptitle(f'{parameter_name} = {p}')

        mw.add_frame()
        plt.close()
    mw.close()

    return mw

## Pupil and focal planes

In the lecture, you learned about pupil and focal planes. Here we'll put use that knowledge to look at a few different pupils.

In [ ]:
pupil_grid = hcipy.make_pupil_grid(512)
focal_grid = hcipy.make_focal_grid(8, 16)

prop = hcipy.FraunhoferPropagator(pupil_grid, focal_grid)

def pupil_focal_planes(obscuration=0, num_spiders=4, spider_width=0):
    ap = hcipy.make_obstructed_circular_aperture(1, obscuration, num_spiders, spider_width)
    pup = ap(pupil_grid)

    wf = hcipy.Wavefront(pup)

    psf = prop(wf).power
    psf /= psf.max()

    plt.subplot(1,2,1)
    hcipy.imshow_field(pup, cmap='gray')
    plt.xlabel('x / D')
    plt.ylabel('y / D')
    plt.subplot(1,2,2)
    hcipy.imshow_field(np.log10(psf), cmap='inferno', vmax=0, vmin=-6)

In [ ]:
make_animation(pupil_focal_planes, 'obscuration', np.linspace(0, 0.99, 100))

In [ ]:
make_animation(pupil_focal_planes, 'num_spiders', range(8), spider_width=0.01)

In [ ]:
make_animation(pupil_focal_planes, 'spider_width', np.linspace(0, 0.1, 51), num_spiders=4)

### Questions

1. Explain what the central obscuration parameter does to your image.
2. Set the number of spiders to 4. Explain what the spider width does to your PSF.
3. Explain what the number of spiders does to your PSF. How many stripes do you see? Does this explain what you expect from what you heard about Fourier optics this morning?

## Adding a speckle

Let's fix the parameters of our pupil and explore what aberrations do to our focal-plane image. To simplify interpretation of the images, we limit outselves at first to sinusoidal aberrations.

In [ ]:
obscuration = 0.1
num_spiders = 6
spider_width = 0.01

ap = hcipy.make_obstructed_circular_aperture(1, obscuration, num_spiders, spider_width)
pup = ap(pupil_grid)

In [ ]:
pos_x = 10
pos_y = 0
amplitude = 0.1
phase = 0

aberration = amplitude * np.cos(2 * np.pi * (pos_x * pupil_grid.x + pos_y * pupil_grid.y) + phase)
aberration = hcipy.Field(aberration, pupil_grid)

hcipy.imshow_field(aberration, cmap='RdBu', mask=pup)
plt.colorbar().set_label('Phase [rad]')
plt.show()

### Bonus question

Can you predict what the focal-plane is going to look like with this phase aberration on the pupil? Hint: what is the Fourier transform of a sinusoid?

In [ ]:
def speckle(pos_x=8, pos_y=0, amplitude=0.3, phase=0):
    aberration = amplitude * np.cos(2 * np.pi * (pos_x * pupil_grid.x + pos_y * pupil_grid.y) + phase)

    wf = hcipy.Wavefront(pup)

    wf.electric_field *= np.exp(1j * aberration)

    psf = prop(wf).power
    psf /= psf.max()

    plt.subplot(1,2,1)
    hcipy.imshow_field(wf.phase, cmap='RdBu', mask=pup, vmin=-1, vmax=1)
    plt.xlabel('x / D')
    plt.ylabel('y / D')
    plt.colorbar()
    plt.subplot(1,2,2)
    hcipy.imshow_field(np.log10(psf), cmap='inferno', vmax=0, vmin=-6)

### Questions

1. What does the amplitude of the sinusoidal aberration do in the focal plane?
2. What does the pos_x parameter do to the sinusoidal aberration? What effect does that have on the two speckles in the focal plane?
3. What does the phase parameter do to the sinusoidal aberration? What effect does that have on the appearance of the two speckles in the focal plane?
4. Bonus: can you find a qualitative explanation for your answer to question 3? Hint: think of even and odd functions, and their Fourier transform.

In [ ]:
make_animation(speckle, 'amplitude', np.linspace(0, 1, 21))

In [ ]:
make_animation(speckle, 'pos_x', np.linspace(0, 10, 21))

In [ ]:
make_animation(speckle, 'phase', np.linspace(0, 2 * np.pi, 51), pos_x=10)

## Light through a vortex coronagraph

Now that you've gotten the hang of imaging, let's step up the difficulty and continue with coronagraphy. We'll use the vortex coronagraph for our example, but there are many other coronagraphs implemented in HCIPy.

### Set up

We'll start by creating the masks of the vortex coronagraph. We'll slightly enlarge the diameter of our pupil grid, to more clearly be able to see the effects of the vortex coronagraph in the Lyot stop.

We'll use a charge 2 vortex coronagraph. Charge 2 means that the phase in the focal-plane mask goes from 0 to $2\pi$ twice azimuthally. Charges for vortex coronagraphs are even, so the charge can be 2, 4, 6, etc.

In [ ]:
pupil_diameter = 1
charge = 2
lyot_mask_diameter = 0.95

pupil_grid = hcipy.make_pupil_grid(1024, diameter=2)
prop = hcipy.FraunhoferPropagator(pupil_grid, focal_grid)

pupil = hcipy.evaluate_supersampled(hcipy.make_circular_aperture(pupil_diameter), pupil_grid, 4)

vortex = hcipy.VortexCoronagraph(pupil_grid, charge=charge)

lyot_mask = hcipy.evaluate_supersampled(hcipy.make_circular_aperture(lyot_mask_diameter), pupil_grid, 4)
lyot_stop = hcipy.Apodizer(lyot_mask)

And we'll show each of the relevant masks here.

In [ ]:
plt.figure(figsize=(12, 3))

plt.subplot(1, 3, 1)
plt.title('Pupil')
hcipy.imshow_field(pupil, cmap='gray')
plt.xlabel('x / D')
plt.ylabel('y / D')

plt.subplot(1, 3, 2)
plt.title('Focal-plane mask')
hcipy.imshow_field(np.exp(1j * charge * focal_grid.as_('polar').theta), focal_grid, vmin=0)
plt.xlabel('x [$\lambda/D$]')
plt.ylabel('y [$\lambda/D$]')

plt.subplot(1, 3, 3)
plt.title('Lyot stop')
hcipy.imshow_field(lyot_mask, cmap='gray')
plt.xlabel('x / D')
plt.ylabel('y / D')

plt.show()

A common way to normalize the images is to normalize by the peak flux of an off-axis source at large angular separation. A quick way to calculate this PSF is to propagate light through the coronagraph without the focal-plane mask.

In [ ]:
wf = hcipy.Wavefront(pupil)
img_ref = prop(lyot_stop(wf)).intensity

hcipy.imshow_field(np.log10(img_ref / img_ref.max()), cmap='inferno', vmin=-5)
plt.show()

Let's now propagate light through each of the planes of the coronagraph. For clarity, we'll show the light at the FPM as well, though this is usually calculated internally in HCIPy for extra accuracy.

In [ ]:
wf = hcipy.Wavefront(pupil)
fpm_img = prop(wf)
pre_lyot_plane = vortex(wf)
post_lyot_plane = lyot_stop(pre_lyot_plane)
science_img = prop(post_lyot_plane)

And then we can show them. Let's wrap everything in a function that propagates a wavefront then plots all planes for convenience.

In [ ]:
def plot_vortex_coronagraph_planes(wf):
    fpm_img = prop(wf)
    pre_lyot_plane = vortex(wf)
    post_lyot_plane = lyot_stop(pre_lyot_plane)
    science_img = prop(post_lyot_plane)

    plt.figure(figsize=(8, 6))

    plt.subplot(2, 3, 1)
    plt.title('At pupil')
    hcipy.imshow_field(wf.intensity, cmap='inferno')
    plt.xlabel('x / D')
    plt.ylabel('y / D')

    plt.subplot(2, 3, 2)
    plt.title('At FPM')
    hcipy.imshow_field(np.log10(fpm_img.intensity / img_ref.max()), vmin=-5, vmax=0, cmap='inferno')
    plt.plot(0, 0, 'w+')
    plt.xlabel('x [$\lambda/D$]')
    plt.ylabel('y [$\lambda/D$]')

    plt.subplot(2, 3, 3)
    plt.title('At Lyot stop')
    hcipy.imshow_field(pre_lyot_plane.intensity, cmap='inferno', vmax=2)
    plt.xlabel('x / D')
    plt.ylabel('y / D')

    plt.subplot(2, 3, 4)
    plt.title('After Lyot stop')
    hcipy.imshow_field(post_lyot_plane.intensity, cmap='inferno', vmax=2)
    plt.xlabel('x / D')
    plt.ylabel('y / D')

    plt.subplot(2, 3, 5)
    plt.title('At science camera')
    hcipy.imshow_field(np.log10(science_img.intensity / img_ref.max()), vmin=-5, vmax=0, cmap='inferno')
    plt.xlabel('x [$\lambda/D$]')
    plt.ylabel('y [$\lambda/D$]')

wf = hcipy.Wavefront(pupil)
plot_vortex_coronagraph_planes(wf)
plt.show()

### Off-axis sources

While suppressing on-axis light is one of the critical parameters of a coronagraph, it is not the only one that matters. Off-axis throughput is just as important, if not more. Let's calculate the off-axis transmission of the vortex coronagraph.

Before that though, let's first see how light from off-axis sources propagate through the vortex coronagraph. We'll use a source at 1.5 $\lambda/D$ to start with.

In [ ]:
wf_offaxis = hcipy.Wavefront(pupil * np.exp(2j * np.pi * pupil_grid.x * 1.5))

# SOLUTION
plot_vortex_coronagraph_planes(wf_offaxis)
plt.show()

### Questions

1. Make an animation of all planes in the vortex coronagraph with the source moving from 0 to 10 $\lambda/D$ angular separation.
2. Off-axis sources located towards the left-right seems to have the light pushed towards the top/bottom in the Lyot plane. Can you find a qualitative explanation for this?
3. Increase the charge of the vortex mask to 4, 6, 8, etc... Explain qualitatively what the charge does to the images.
4. Bonus: what does the image of a vortex coronagraph with odd charges look like?

In [ ]:
def plot_vortex_coronagraph_offaxis(offaxis_angle):
    wf = hcipy.Wavefront(pupil * np.exp(2j * np.pi * pupil_grid.x * offaxis_angle))
    plot_vortex_coronagraph_planes(wf)

make_animation(plot_vortex_coronagraph_offaxis, 'offaxis_angle', np.linspace(0, 10, 41))

Let's now look at the throughput of the off-axis source. One of the current favourite ways of computing throughput is using aperture photometry. The choice of aperture is however still a hotly debated topic. In this session, we will investigate only one. This throughput metric uses a photometric aperture defined by the contour of half the peak flux of the coronagraphic image.

[NEEDS IMAGE FOR EXPLANATION]

The throughput is then defined as the total power inside that aperture divided by the incident power on the telescope. All propagations in HCIPy will conserve power, unless blocked by that optical element. This means that we can simply prepare an incoming wavefront with a total power of 1, compute the coronagraphic image, and then sum all flux inside the photometric aperture.

Let's do that now.

In [ ]:
angular_separations = np.linspace(0, 10, 101)
throughputs = []

for angular_separation in tqdm(angular_separations):
    wf = hcipy.Wavefront(pupil * np.exp(2j * np.pi * pupil_grid.x * angular_separation))
    wf.total_power = 1

    pre_lyot_plane = vortex(wf)
    post_lyot_plane = lyot_stop(pre_lyot_plane)
    science_img = prop(post_lyot_plane)

    photometric_aperture = science_img.power > (0.5 * science_img.power.max())

    throughput = (science_img.power * photometric_aperture).sum()
    throughputs.append(throughput)

In [ ]:
plt.plot(angular_separations, throughputs)
plt.xlabel('Angular separation [$\lambda/D$]')
plt.ylabel('Core throughput')
plt.show()

### Questions

1. Write code to compute the core throughput.
2. The core throughput doesn't asymptotically go to one. Why shouldn't it go to one?
3. Compute the maximum core throughput at infinite angular separation. Hint: we did something like this before when computing the reference PSF.
4. Compute the inner working angle (the angular separation where the core throughput reaches half of the maximum core throughput) of the charge 2 vortex coronagraph.
4. The core throughput looks very jagged. Can you give a reason why? What could you improve in the code to reduce this jaggedness?
5. Plot the core throughput of higher charges (charge 4, 6, 8, etc...) in the same plot. What does this look like? How does inner working angle change?

## Aberrations

You already saw how sinusoidal aberrations create single speckles in the focal plane. While sinusoidal aberrations provide an excellent basis to understanding interference, it does not describe the type of aberrations typically seen in coronagraphic systems. Typically, coronagraphs suppress certain aberrations more than others. In particular, low-order aberrations are passively suppressed by the coronagraph, which impacts requirements on the adaptive optics system. In addition, low-order aberrations are usually much stronger in coronagraphic systems since misalignments and drifts primarily create low-order compared to high-order aberrations.

When evaluating the performance of coronagraphs, we therefore often observe the passive suppression of low-order aberrations more than high-order aberrations. A commonly-used basis, especially for vortex coronagraphs, is the Zernike basis. Zernike modes are implemented in HCIPy. Let's display them first, before working with them.

In [ ]:
def plot_zernike(noll_index):
    zernike_indices = hcipy.noll_to_zernike(noll_index)
    mode = hcipy.zernike(*zernike_indices)(pupil_grid)

    vlim = np.pi

    hcipy.imshow_field(mode, cmap='RdBu', mask=pupil, vmin=-vlim, vmax=vlim)
    plt.xlabel('x / D')
    plt.ylabel('y / D')
    plt.xlim(-0.75, 0.75)
    plt.ylim(-0.75, 0.75)
    plt.colorbar()

In [ ]:
make_animation(plot_zernike, 'noll_index', range(1, 22))

Ruane (2018) is an excellent introduction on the behaviour of Zernike aberrations through the vortex coronagraph. Let's recreate the first few figures of this paper.

In [ ]:
zernike_modes = [
    hcipy.zernike(0, 0),
    hcipy.zernike(2, 0),
    hcipy.zernike(2, 2),
    hcipy.zernike(3, 1),
    hcipy.zernike(3, 3)
]

zernike_labels = [
    'Piston',
    'Defocus',
    'Astig.',
    'Coma',
    'Trefoil'
]

charges = [2, 4, 6, 8, 10]

n = len(charges) + 1
m = len(zernike_modes)

plt.figure(figsize=(6.7, 8))
i = 0

for l, zernike in enumerate(zernike_modes):
    plt.subplot(n, m, i + 1)

    hcipy.imshow_field(zernike(pupil_grid), cmap='RdBu', vmin=-np.pi, vmax=np.pi)

    plt.gca().xaxis.set_ticks([])
    plt.gca().yaxis.set_ticks([])

    if l == 0:
        plt.ylabel('Entrance')

    plt.title(zernike_labels[l])

    i += 1

for k, charge in enumerate(charges):
    vortex = hcipy.VortexCoronagraph(pupil_grid, charge=charge)

    for l, zernike in enumerate(zernike_modes):
        wf = hcipy.Wavefront(zernike(pupil_grid))

        lyot = vortex(wf)

        plt.subplot(n, m, i + 1)
        hcipy.imshow_field(lyot.amplitude, vmax=1.5, cmap='inferno')

        plt.gca().xaxis.set_ticks([])
        plt.gca().yaxis.set_ticks([])

        if l == 0:
            plt.ylabel(f'Charge {charge}')

        i += 1

plt.subplots_adjust(wspace=0.02, hspace=0.02)
plt.show()

In [ ]:
scoring_region = hcipy.make_circular_aperture(3.5 * 2)(focal_grid) - hcipy.make_circular_aperture(2.5 * 2)(focal_grid)
scoring_region = scoring_region > 0.5

vortex = hcipy.VortexCoronagraph(pupil_grid, charge=6)
coro = hcipy.PerfectCoronagraph(pupil)

noll_indices = [2, 4, 5, 7, 10, 9, 12, 17, 19, 23, 25, 34, 32, 28, 37]
zernike_labels = [
    'tip-tilt',
    'defocus',
    'astig.',
    'coma',
    'trefoil',
    'spher.',
    '2nd astig.',
    'quadrafoil',
    '2nd coma',
    '2nd tref.',
    'pentafoil',
    '2nd spher.',
    '3rd astig.',
    '2nd quad.',
    'hexafoil'
]

coeffs = np.linspace(1e-6, 1e0, 7)

for k, noll_index in tqdm(enumerate(noll_indices)):
    aberration = hcipy.zernike(*hcipy.noll_to_zernike(noll_index), radial_cutoff=False)(pupil_grid)

    mean_norm_irradiances = []

    for c in coeffs:
        wf = hcipy.Wavefront(pupil * np.exp(2j * np.pi * c * aberration))
        wf = coro(wf)

        pre_lyot_plane = vortex(wf)
        post_lyot_plane = lyot_stop(pre_lyot_plane)
        science_img = prop(post_lyot_plane)

        norm_irradiance = science_img.intensity / img_ref.max()
        mean_norm_irradiance = np.mean(norm_irradiance[scoring_region])

        #hcipy.imshow_field(np.log10(norm_irradiance * scoring_region), vmin=-11, vmax=-0)
        #plt.show()

        mean_norm_irradiances.append(mean_norm_irradiance)

    plt.plot(coeffs, mean_norm_irradiances, '-', label=zernike_labels[k])

plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-6, 1e0)
plt.ylim(5e-12, 2e-9)
plt.grid()
plt.legend()
plt.show()

# Fast aberrations

Fast aberrations are any aberrations that vary on timescales faster than the integration time of the atmosphere. Counter to slow aberrations (ie. those that are slower than the integration time of the camera), these aberrations appear as incoherent light.

* Perform